# AI Search initialization

## Prerequisites
Configure a Python virtual environment for 3.10 or later: 
 1. open the Command Palette (Ctrl+Shift+P).
 1. Search for Python: Create Environment.
 1. select Venv / Conda and choose where to create the new environment.
 1. Select the Python interpreter version. Create with version 3.10 or later.

For a dependency installation, run the code below to install the packages required to run it. 

```bash
# Create a virtual environment
python -m venv venv

# Activate the virtual environment
# On Windows
venv\Scripts\activate

# On macOS/Linux
source venv/bin/activate

pip install -r requirements.txt
```

## Set up your environment
Git clone the repository to your local machine. 

```bash
git clone https://github.com/hyogrin/Azure_OpenAI_samples.git
```

Create an .env file based on the .env-sample file. Copy the new .env file to the folder containing your notebook and update the variables.

## Load environment variables

In [16]:

import os

import json
from openai import AzureOpenAI
import sys
import pandas as pd
import tiktoken
import re
from dotenv import load_dotenv

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import VectorizableTextQuery, QueryType
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    AzureOpenAIVectorizer,
    SemanticConfiguration,
    SemanticSearch,
    SemanticPrioritizedFields,
    AzureOpenAIVectorizerParameters,
    SemanticField,
    SearchIndex,
    VectorSearchAlgorithmMetric,
    VectorSearchAlgorithmKind,
    HnswParameters,
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    ScoringProfile,
    TextWeights,
)

from tenacity import retry, wait_random_exponential, stop_after_attempt

load_dotenv("../app/backend/.env", override=True)

search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
index_name = os.getenv("AZURE_SEARCH_INDEX")
admin_key = os.getenv("AZURE_SEARCH_API_KEY")
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT").replace("wss://", "https://")
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
small3_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
gpt_chat_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT")

print(f"search_endpoint: {search_endpoint}")

search_endpoint: https://rag-innovator-search-svc.search.windows.net


## Create embeddings
- Insert Azure AI Search indexes by reading data, creating OpenAI embeddings, and exporting to a valid format

In [4]:
MAX_RETRIES = 3
client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),        # Azure OpenAI base URL
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("AZURE_OPENAI_CHAT_API_VERSION"),
    max_retries    = MAX_RETRIES
)

In [5]:
!ls data/rag_sample_data_ko.jsonl -lh

-rw-r--r-- 1 azureuser azureuser 118K May  9 01:49 data/rag_sample_data_ko.jsonl


In [6]:
# Generate Document Embeddings using OpenAI Small 3

# Read the text-sample.json
# with open("text-sample.json", "r", encoding="utf-8") as file:
#     input_data = json.load(file)

df_input_data = pd.read_json(os.path.join(os.getcwd(),'data/rag_sample_data_ko.jsonl'), lines=True)    

In [7]:
#pd.options.mode.chained_assignment = None #https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#evaluation-order-matters

# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df_input_data['content']= df_input_data["content"].apply(lambda x : normalize_text(x))

- To take advantage of the Embedding API provided by Azure OpenAI, we check that the document does not have more than 8,192 tokens of text in the document

In [8]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df_input_data['n_tokens'] = df_input_data["content"].apply(lambda x: len(tokenizer.encode(x)))
df_input_data = df_input_data[df_input_data.n_tokens<8192]
len(df_input_data)
df_input_data

,id,category,type,title,content,n_tokens
0,1,General,Whats New,X10 Series General Whats New,"스타즈의 최신 블랙홀 X10 시리즈는 최신 기술을 집약한 스마트폰으로, 다양한 AI...",766
1,2,Design,Color,X10 Series Design Color,"블랙홀 X10 시리즈는 자연에서 영감을 받은 클래식한 컬러들로 구성되어 있어, 사용...",342
2,3,Design,Design,X10 Series Design Design,"블랙홀 스마트폰은 독특한 시그니처 디자인으로 잘 알려져 있으며, 이는 브랜드의 정체...",357
3,5,Design,Color,X10 Series Design Color,스타즈닷컴에서는 다양한 전용 컬러 옵션을 제공하여 소비자들이 개인의 취향에 맞는 제...,354
4,6,Design,Materials,X10 Series Design Materials,재활용 금속 프레임은 환경 친화적인 소재로 제작되어 지속 가능한 소비를 지향하는 현...,264
...,...,...,...,...,...,...
99,95,Display,Dynamic AMOLED 2X,X10 FE Display Dynamic AMOLED 2X,블랙홀 X10 FE는 전작보다 더 커진 6.7인치 FHD+ 디스플레이를 탑재하여 사...,492
100,96,Display,Brightness,X10 FE Display Brightness,블랙홀 X10 FE의 Dynamic AMOLED 2X 디스플레이는 최신 기술을 적용...,411
101,97,Display,Size,X10 FE Display Size,새로운 블랙홀 X10 FE는 6.7인치 FHD+ 디스플레이를 탑재하여 사용자에게 선...,321
102,98,Blackhole Ecosystem,Link to Windows,X10 FE Blackhole Ecosystem Link to Windows,블랙홀 스마트폰과 PC 간의 연속성 기능이 더욱 강화되었습니다. 이 기능은 사용자들...,407


- After completing the verification, delete the columns that are no longer needed, and save the data for insertion into Azure AI Search. 

In [9]:
df_input_data = df_input_data.drop('n_tokens', axis=1)

In [10]:
# Generate Document Embeddings using OpenAI Small 3
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for title and content fields, also used for query embeddings
def get_embedding(text, model=small3_deployment): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [11]:
# model should be set to the deployment name you chose when you deployed the text-embedding-small3 (Version 2) model
df_input_data['content_vector'] = df_input_data["content"].apply(lambda x : get_embedding (x, model = small3_deployment)) 
df_input_data

,id,category,type,title,content,content_vector
0,1,General,Whats New,X10 Series General Whats New,"스타즈의 최신 블랙홀 X10 시리즈는 최신 기술을 집약한 스마트폰으로, 다양한 AI...","[0.027465613558888435, 0.0006100021419115365, ..."
1,2,Design,Color,X10 Series Design Color,"블랙홀 X10 시리즈는 자연에서 영감을 받은 클래식한 컬러들로 구성되어 있어, 사용...","[0.05609920248389244, -0.00802561640739441, -0..."
2,3,Design,Design,X10 Series Design Design,"블랙홀 스마트폰은 독특한 시그니처 디자인으로 잘 알려져 있으며, 이는 브랜드의 정체...","[0.03267679363489151, -0.010334414429962635, -..."
3,5,Design,Color,X10 Series Design Color,스타즈닷컴에서는 다양한 전용 컬러 옵션을 제공하여 소비자들이 개인의 취향에 맞는 제...,"[0.04042738676071167, -0.038710471242666245, -..."
4,6,Design,Materials,X10 Series Design Materials,재활용 금속 프레임은 환경 친화적인 소재로 제작되어 지속 가능한 소비를 지향하는 현...,"[0.021915271878242493, 0.00026288023218512535,..."
...,...,...,...,...,...,...
99,95,Display,Dynamic AMOLED 2X,X10 FE Display Dynamic AMOLED 2X,블랙홀 X10 FE는 전작보다 더 커진 6.7인치 FHD+ 디스플레이를 탑재하여 사...,"[0.023891305550932884, -0.011532075703144073, ..."
100,96,Display,Brightness,X10 FE Display Brightness,블랙홀 X10 FE의 Dynamic AMOLED 2X 디스플레이는 최신 기술을 적용...,"[0.027800427749753, 0.022450517863035202, -0.0..."
101,97,Display,Size,X10 FE Display Size,새로운 블랙홀 X10 FE는 6.7인치 FHD+ 디스플레이를 탑재하여 사용자에게 선...,"[0.04390879347920418, -0.022210104390978813, -..."
102,98,Blackhole Ecosystem,Link to Windows,X10 FE Blackhole Ecosystem Link to Windows,블랙홀 스마트폰과 PC 간의 연속성 기능이 더욱 강화되었습니다. 이 기능은 사용자들...,"[0.024208439514040947, 0.013966407626867294, -..."


In [12]:
df_input_data.to_csv(os.path.join(os.getcwd(),'data/embedding_input_data.csv'), index=False)

## Create your search index
Create your search index schema and vector search configuration:

In [17]:
# Create a search index
credential = AzureKeyCredential(admin_key)

index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)
fields = [
        SearchableField(name="id", key=True, type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=False),  
        SearchableField(name="category", type=SearchFieldDataType.String, filterable=True, facetable=True),
        SearchableField(name="type", type=SearchFieldDataType.String, filterable=True, facetable=True),
        SearchableField(name="title", type=SearchFieldDataType.String, filterable=True, facetable=True),
        SearchableField(name="content", type=SearchFieldDataType.String, filterable=True, facetable=True, analyzer_name="ko.microsoft"),
        SearchField(
            name="content_vector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            vector_search_dimensions=1536,
            vector_search_profile_name="myHnswProfile",  # Ensure vector_search_profile is set
        ),
        ]
# Configure the vector search configuration  
vector_search = VectorSearch(  
    algorithms=[  
        HnswAlgorithmConfiguration(
            name="myHnsw",
            kind=VectorSearchAlgorithmKind.HNSW,
            parameters=HnswParameters(
                m=10,
                ef_construction=1000,
                ef_search=500,
                metric=VectorSearchAlgorithmMetric.COSINE
            )
        ),
        ExhaustiveKnnAlgorithmConfiguration(
            name="myExhaustiveKnn",
            kind=VectorSearchAlgorithmKind.EXHAUSTIVE_KNN,
            parameters=ExhaustiveKnnParameters(
                metric=VectorSearchAlgorithmMetric.COSINE
            )
        )
    ],  
    profiles=[  
        VectorSearchProfile(  
            name="myHnswProfile",  
            algorithm_configuration_name="myHnsw",  
            vectorizer_name="myOpenAI_vectorizer",  
        ),
        VectorSearchProfile(
            name="myExhaustiveKnnProfile",
            algorithm_configuration_name="myExhaustiveKnn",
            vectorizer_name="myOpenAI_vectorizer",  
        )
        
    ],  
    vectorizers=[  
        AzureOpenAIVectorizer(  
            kind="azureOpenAI",  
            vectorizer_name="myOpenAI_vectorizer",  # Added the missing argument
            parameters=AzureOpenAIVectorizerParameters(  
                resource_url=openai_endpoint,  
                deployment_name=small3_deployment,
                model_name=small3_deployment,
                api_key=openai_api_key,
            ),
            
        ),  
    ],  
)  


    
  
semantic_config = SemanticConfiguration(  
    name="my-semantic-config",  
        prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        content_fields=[SemanticField(field_name="content")]  
    ),  
)

# New scoring profile for keyword search
# https://learn.microsoft.com/en-us/python/api/azure-search-documents/azure.search.documents.indexes.models.scoringprofile?view=azure-python
scoring_profiles = [  
    ScoringProfile(  
        name="my-scoring-profile",
        text_weights = TextWeights(
        	weights = {
                'category' : 1,
                'type' : 1,
                'title' : 3,
                'content' : 1
            }
        )
    ),
]


# Create the semantic search with the configuration  
semantic_search = SemanticSearch(configurations=[semantic_config])  

# Create a suggester that is a configuration in an index that specifies which fields should be used to populate autocomplete and suggested matches.
suggesters= [
    {
        "name": "sg",
        "searchMode": "analyzingInfixMatching",
        "sourceFields": ["title"]
    }
]

try:
    existing_index = index_client.get_index(index_name)
    print(f"index '{index_name}' exists. Proceed the index update.")

    index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search, semantic_search=semantic_search,suggesters=suggesters, scoring_profiles=scoring_profiles)
    result = index_client.create_or_update_index(index)
    print(f"index '{result.name}' updated.")
    

except Exception as e:
    print(f"index ( {index_name} ) does not exist. Create a new index.")

    index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search, semantic_search=semantic_search,suggesters=suggesters, scoring_profiles=scoring_profiles)
    result = index_client.create_or_update_index(index)
    print(f"index ( {result.name} ) creation completed.")
    

index 'optimize-test-index' exists. Proceed the index update.
index 'optimize-test-index' updated.


## Insert text and embeddings into vector store
Add texts and metadata from the JSON data to the vector store:

In [18]:
# Upload data_embeddings.csv to search_client.upload_documents to create the index 
documents = df_input_data.to_dict(orient='records')
for doc in documents:
    doc['id'] = str(doc['id'])
search_client = SearchClient(
    endpoint=search_endpoint, index_name=index_name, credential=credential
)
result = search_client.upload_documents(documents)

print(f"Uploaded {len(documents)} documents")

Uploaded 104 documents


In [19]:
vector_queries = []
vector_queries.append(VectorizableTextQuery(text='x10', k_nearest_neighbors=3, fields='content_vector'))

search_results = search_client.search(
    search_text='X10', 
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name="my-semantic-config",
    top=5,
    select=["title","category","content"],
    vector_queries=vector_queries
)

In [20]:
for result in search_results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

Title: X10 Series AI Whats New
Score: 0.014925372786819935
Content: 스타즈의 첫 번째 진정한 블랙홀 AI폰, X10 시리즈를 소개합니다! 이 시리즈는 최신 인공지능 기술을 활용하여 사용자 경험을 혁신적으로 개선합니다. 특히, AI 기능 중 하나인 자동 빈 공간 메우기 기능은 사진 편집에서 큰 변화를 가져옵니다. 사용자가 사진을 촬영할 때, 원치 않는 요소나 빈 공간이 발생할 수 있는데, X10 시리즈의 AI는 이러한 빈 공간을 자동으로 인식하고 주변 환경에 맞게 자연스럽게 메워줍니다. 이 기능은 단순한 이미지 보정에 그치지 않고, 사용자가 더 창의적인 작업을 할 수 있도록 도와줍니다. 예를 들어, 여행 중 찍은 사진에서 불필요한 배경 요소를 제거하고, 그 자리를 주변 풍경과 조화롭게 연결함으로써 더욱 완성도 높은 이미지를 생성할 수 있습니다. 또한, X10 시리즈는 고급 카메라 시스템과 결합되어 다양한 촬영 모드와 필터를 제공하며, AI는 사용자의 촬영 스타일을 학습하여 최적의 설정을 자동으로 추천합니다. 이러한 혁신적인 기능들은 사진 촬영과 편집의 경계를 허물고, 모든 사용자가 전문가 수준의 결과물을 쉽게 얻을 수 있도록 돕습니다. X10 시리즈는 단순한 스마트폰을 넘어, 개인의 창의성을 극대화하는 도구로 자리잡고 있습니다.
Category: AI

Title: X10 Series AI Whats New
Score: 0.03279569745063782
Content: X10 시리즈는 최신 기술을 활용하여 언제 어디서나 번역이 필요한 순간에 완벽한 도우미 역할을 수행합니다. 이 기기는 실시간 번역 기능을 제공하여 사용자가 다양한 언어로 된 콘텐츠를 즉시 이해할 수 있도록 돕습니다. 예를 들어, 해외 여행 중 현지인과의 대화, 외국어로 작성된 문서의 이해, 또는 국제 비즈니스 회의에서의 원활한 소통을 지원합니다. X10 시리즈는 다양한 플랫폼에서 작동하며, 스마트폰, 태블릿, 노트북 등 여러 기기와 호환되어 